In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Wed Jul 13 13:57:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive/
!ls /mydrive

Mounted at /content/gdrive
ln: target '/mydrive/' is not a directory: No such file or directory
ls: cannot access '/mydrive': No such file or directory


In [3]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15457, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 15457 (delta 4), reused 21 (delta 2), pack-reused 15424
Receiving objects: 100% (15457/15457), 14.12 MiB | 20.68 MiB/s, done.
Resolving deltas: 100% (10367/10367), done.


In [4]:
# Configure
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [5]:
# Compile
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

In [6]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [7]:
# Change lines in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [8]:
!echo -e 'Cylinders' > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /content/gdrive/MyDrive/yolov3' > data/obj.data

In [9]:
!mkdir data/obj
# !unzip MyDrive/yolov3/dataset.zip -d data/obj
!unzip /content/gdrive/MyDrive/yolov3/dataset.zip -d data/obj

Archive:  /content/gdrive/MyDrive/yolov3/dataset.zip
  inflating: data/obj/dataset/README.dataset.txt  
  inflating: data/obj/dataset/README.roboflow.txt  
  inflating: data/obj/dataset/test/2021_08_12_10_14_01_5420_jpg.rf.c08f6b94bea50e882d6332bbdc3524c5.jpg  
  inflating: data/obj/dataset/test/2021_08_10_16_27_37_10568_jpg.rf.e6679de846ee1daec76ac54cc0319aba.jpg  
  inflating: data/obj/dataset/test/2021_08_10_13_42_28_6900_jpg.rf.27bfed288e889985759434b4edc7d8a5.jpg  
  inflating: data/obj/dataset/test/2021_08_10_17_40_13_12321_jpg.rf.e377a2a77a761b65c75cc1859d83c590.jpg  
  inflating: data/obj/dataset/test/2021_08_12_10_14_30_5435_jpg.rf.fab4082b8642ae052bd908941ca8a371.jpg  
  inflating: data/obj/dataset/test/2021_08_10_15_50_28_9799_jpg.rf.b44fe3b788e2438ab7655712d61e8cbe.jpg  
  inflating: data/obj/dataset/test/2021_08_11_17_04_32_4077_jpg.rf.436b0c875f56a78792d62e9fb58fb94f.jpg  
  inflating: data/obj/dataset/test/2021_08_10_12_35_46_4524_jpg.rf.441b283f2f9f52331ca761d56dc9f3aa.

In [10]:
import glob
images_list = glob.glob("data/obj/dataset/train/*.jpg")

# print(' '.join(images_list))
with open("data/train.txt", "w") as f:
    print(' '.join(images_list))
    f.write("\n".join(images_list))

data/obj/dataset/train/2021_02_18_15_55_26_4630_jpg.rf.dc45ecb58febeacac4615aa7bf5c3790.jpg data/obj/dataset/train/2021_08_08_11_46_20_991_jpg.rf.5dbef9360def68f1446333a8b259a868.jpg data/obj/dataset/train/2021_02_19_19_21_21_13341_jpg.rf.d63fe302ae1e3c66075601eccd574681.jpg data/obj/dataset/train/2021_07_06_13_02_58_8978_jpg.rf.49ce1fc6299336a7f0f2d08da6b7a031.jpg data/obj/dataset/train/2021_08_11_15_40_08_2253_jpg.rf.eb28e4137a02d8e055dac11b47572cb5.jpg data/obj/dataset/train/2021_03_03_06_17_00_403_jpg.rf.38269c42dab3e9243455b10cfb1e62d0.jpg data/obj/dataset/train/2021_08_13_10_15_09_1440_jpg.rf.08634975b727715f1b0a8b0f54492e32.jpg data/obj/dataset/train/2021_02_25_19_57_41_8767_jpg.rf.c0271b3a53363eb65fc5671adceb53ee.jpg data/obj/dataset/train/2021_02_14_16_21_11_6068_jpg.rf.e6f5fcf508e550884b1ec1045e4a75de.jpg data/obj/dataset/train/2021_08_12_07_08_33_1149_jpg.rf.b65686fc8d115f2e9d3dbc87e7e14d95.jpg data/obj/dataset/train/2021_03_09_18_40_32_6702_jpg.rf.39b8a54f62aaf479e8ff6213bb

In [11]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-07-13 13:59:21--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  59.9MB/s    in 2.6s    

2022-07-13 13:59:24 (59.9 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
# Uncomment below and comment above to re-start your training from last saved weights
# !./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 134817, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.857871), count: 4, class_loss = 0.082894, iou_loss = 0.092162, total_loss = 0.175056 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 134821, rewritten_bbox = 0.000000 % 
]2;2127/4000: loss=0.0 hours left=3.5
 2127: 0.037261, 0.059858 avg loss, 0.001000 rate, 10.113759 seconds, 136128 images, 3.534030 hours left
Loaded: 0.000067 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.847496), count: 4, class_loss = 0.214497, iou_loss = 0.113267, total_loss = 0.327764 
v3 (